In [ ]:
import ROOT
import numpy as np
from hepdata_lib import Submission, Table, Variable, RootFileReader, Uncertainty
import ROOT as rt
import os

In [76]:
submission = Submission()

## Figure 1 - 3 Postfit Data - MC

In [77]:
chgs = ["+", "-", "z"]
energies = ["5020.0", "13000.0"]

chans = {
    "+": ["we", "wmu"],
    "-": ["we", "wmu"],
    "z": ["ee", "mumu"]
}

def GetChgString(chg):
    if chg == "z":
        return ""
    return "plus" if chg == "+" else "minus"

def GetEnergyString(energy):
    return "5TeV" if energy == "5020.0" else "13TeV"

def GetChannelString(chan):
    if chan == "ee":
        return "ee"
    if chan == "mumu":
        return "mumu"
    if chan == "we":
        return "e"
    if chan == "wmu":
        return "mu"

def GetFigureIndex(chg):
    return chgs.index(chg) + 1

indices = ["a", "b", "c", "d"]

desc_strs = {}
desc_strs['+'] = "Distributions of $m_T$ in the $W^{+}$ signal selection for CHAN final states \
for the pp collisions at $\\sqrt{s}=$ ENERGY after the maximum likelihood fit. \
The EW backgrounds include the contributions from DY, $W\\to\\tau\\nu$, and diboson processes."
desc_strs['-'] = desc_strs['+'].replace("$W^{+}$", "$W^{-}$")
desc_strs['z'] = "Distributions of $m_ll$ in the Z signal selection for CHAN final states \
for the pp collisions at $\\sqrt{s}=$ ENERGY after the maximum likelihood fit. \
The EW backgrounds include the contributions from DY, $W\\to l\\nu$, and diboson processes."
    

for chg in chgs:
    idx = 0
    for energy in energies:
        for chan in chans[chg]:
            fig_str = "Figure " + str(GetFigureIndex(chg)) + "-"
            fig_str += indices[idx]
            idx += 1
            
            table = Table(fig_str)
            table.location = fig_str
            
            chg_str = GetChgString(chg)
            energy_str = GetEnergyString(energy)
            chan_str = GetChannelString(chan)
            
            desc = desc_strs[chg].replace("CHAN", chan_str).replace("ENERGY", energy_str)
            print("descriptions: ", desc)
            table.description = desc
            table.keywords["cmenergies"] = [energy]
            if chg != "z":
                table.keywords["observables"] = ["mT"]
            else:
                table.keywords["observables"] = ["mll"]
            
            if chg == "z":
                fname_str = f"{chan_str}__mT0_{energy_str}"
                fname = f"root/histo_zjets_{fname_str}.root"
            else:
                fname_str = f"{chan_str}{chg_str}__mT0_{energy_str}"
                fname = f"root/histo_wjets_{fname_str}.root"
            print(fname)
            
            reader = RootFileReader(fname)
            QCD = reader.read_hist_1d(f"hqcd_{fname_str}")
            TT = reader.read_hist_1d(f"httbar_{fname_str}")
            EWK = reader.read_hist_1d(f"hewk_{fname_str}")
            SIG = reader.read_hist_1d(f"hsig_{fname_str}")
            Data = reader.read_hist_1d(f"hdata_{fname_str}")
            ExpTot = reader.read_hist_1d(f"htot_{fname_str}")
            
            # x and y axis
            if chg != "z":
                xax = Variable("$m_T$", is_independent=True, is_binned=True, units="GeV")
            else:
                xax = Variable("$m_{ll}$", is_independent=True, is_binned=True, units="GeV")
            xax.values = Data["x_edges"]
            yax = Variable("Events", is_independent=False, is_binned=False, units="Events / GeV")
            yax.values = Data["y"]
            
            # read data
            tot = Variable("Number of total expected events after the fit", is_independent=False, is_binned=False, units="Events / GeV")
            tot.values = ExpTot["y"]
            tot.add_qualifier("Process", "Total predicted after the fit")
            qcd = Variable("Number of QCD multijet events", is_independent=False, is_binned=False, units="Events / GeV")
            qcd.values = QCD["y"]
            qcd.add_qualifier("Process", "QCD multijet")
            tt = Variable("Number of ttbar events", is_independent=False, is_binned=False, units="Events / GeV")
            tt.values = TT["y"]
            tt.add_qualifier("Process", "ttbar")
            ewk = Variable("Number of EWK events", is_independent=False, is_binned=False, units="Events / GeV")
            ewk.values = EWK["y"]
            ewk.add_qualifier("Process", "EWK")
            if chg != "z":
                sig = Variable(f"Number of W{chg_str} to {chan_str}{chg_str}nu events", is_independent=False, is_binned=False, units="Events / GeV")
            else:
                sig = Variable(f"Number of Z to {chan_str} events", is_independent=False, is_binned=False, units="Events / GeV")
            sig.values = SIG["y"]
            sig.add_qualifier("Process", "Signal")
            data = Variable("Number of data events", is_independent=False, is_binned=False, units="Events / GeV")
            data.values = Data["y"]
            data.add_qualifier("Process", "Data")
            
            # uncertainties 
            unc_tot = Uncertainty("total postfit uncertainty", is_symmetric=True)
            unc_tot.values = ExpTot["dy"]
            unc_data = Uncertainty("Poisson uncertainty", is_symmetric=True)
            unc_data.values = Data["dy"]
            tot.add_uncertainty(unc_tot)
            data.add_uncertainty(unc_data)
            
            # add variables to table
            table.add_variable(xax)
            table.add_variable(yax)
            table.add_variable(tot)
            if chg != "z":
                table.add_variable(qcd)
            table.add_variable(tt)
            table.add_variable(ewk)
            table.add_variable(sig)
            table.add_variable(data)
            
            submission.add_table(table)

descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for e final states     for the pp collisions at $\sqrt{s}=$ 5TeV after the maximum likelihood fit.     The EW backgrounds include the contributions from DY, $W\to\tau\nu$, and diboson processes.
root/histo_wjets_eplus__mT0_5TeV.root
descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for mu final states     for the pp collisions at $\sqrt{s}=$ 5TeV after the maximum likelihood fit.     The EW backgrounds include the contributions from DY, $W\to\tau\nu$, and diboson processes.
root/histo_wjets_muplus__mT0_5TeV.root
descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for e final states     for the pp collisions at $\sqrt{s}=$ 13TeV after the maximum likelihood fit.     The EW backgrounds include the contributions from DY, $W\to\tau\nu$, and diboson processes.
root/histo_wjets_eplus__mT0_13TeV.root
descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for mu final stat

In [78]:
submission.create_files(remove_old=True, outdir="output")


## Figure 4 - 9 Measured vs Theory Cross Sections